In [1]:
using Pkg


cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv" # 1D Earth/Mars/Moon models are defined

include("../src/commonBatchs.jl")
include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

include("../src/Neurthino.jl")

using .commonBatchs, .planet1D, .GeoPoints
using .Neurthino 
using Colors


  Activating project at `~/Documents/Github/flexOPT`


In [2]:
using SegyIO, CairoMakie

In [ ]:


file = "../dataObs/seismo2026/2.sgy"

# --- IBM float conversion ---
function ibm_to_float32(x::UInt32)
    if x == 0
        return 0.0f0
    end
    sign = (x & 0x80000000) != 0 ? -1.0f0 : 1.0f0
    exponent = Int((x >> 24) & 0x7f) - 64
    mantissa = Float32(x & 0x00ffffff) / 0x01000000
    return sign * mantissa * 16.0f0^exponent
end
function read
open(file, "r") do io
    
    # ---- Read binary header ----
    seek(io, 3200 + 16)
    dt = ntoh(read(io, Int16))
    
    seek(io, 3200 + 20)
    ns = ntoh(read(io, Int16))
    
    seek(io, 3200 + 24)
    format = ntoh(read(io, Int16))
    
    println("dt = ", dt)
    println("ns = ", ns)
    println("format = ", format)

    # ---- Compute number of traces ----
    filesize = stat(file).size
    trace_size = 240 + 4*ns
    ntr = Int((filesize - 3600) ÷ trace_size)
    println("ntr = ", ntr)

    # ---- Read traces ----
    data = zeros(Float32, ns, ntr)

    seek(io, 3600)

    for i in 1:ntr
        read(io, 240)
        raw = read!(io, Vector{UInt32}(undef, ns))

        data[:, i] .= ibm_to_float32.(ntoh.(raw))
    end

    # ---- Time axis ----
    t = (0:ns-1) .* (dt/1e6)

    # ---- Plot gather ----
    fig = Figure(resolution = (900,700))
    ax = Axis(fig[1,1],
        xlabel = "Trace",
        ylabel = "Time (s)",
        yreversed = true)

    maxamp = maximum(abs.(data))
    heatmap!(ax, 1:ntr, t, data;
        colormap = :gray,
        colorrange = (-maxamp, maxamp))

    fig
end


dt = 250
ns = 12000
format = 1
ntr = 216


┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie /Users/nobuaki/.julia/packages/Makie/TOy8O/src/scenes.jl:264


In [4]:
t

UndefVarError: UndefVarError: `t` not defined in `Main`
Suggestion: check for spelling errors or missing imports.